# Import Library

In [1]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Visuzliation Setting
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
from matplotlib import colors
import seaborn as sns

---

In [2]:
root = os.path.join(os.getcwd(), 'DATA')

# Training Data
빅콘테스트 제공 데이터 전처리

## Import Data

In [3]:
df_raw = pd.read_excel(os.path.join(root, 'train.xlsx'))
df_train = df_raw.copy()

### Add Columns

In [4]:
# 날짜변수 추가
df_train['year'] = df_train['REG_DATE'].dt.year
df_train['month'] = df_train['REG_DATE'].dt.month
df_train['day'] = df_train['REG_DATE'].dt.day

In [7]:
df_train['VALUE_COUNT'] = 0;
value_dict = {}
for name, value in zip(df_train['P_NAME'].value_counts().index,df_train['P_NAME'].value_counts()):
    value_dict[name] = value

def value(col):
    return value_dict[col]

df_train['VALUE_COUNT'] = df_train['P_NAME'].apply(value)

### One hot encoding
P_IMPORT_TYPE 이라는 특수한 컬럼에 대한 전처리

In [5]:
import_type_list = set()
for tmp in df_train.P_IMPORT_TYPE.unique():
    for a in tmp.split(','):
        import_type_list.add(a)

In [6]:
for name in import_type_list:
    df_train[name] = 0
    df_train.loc[df_train['P_IMPORT_TYPE'].str.contains(name, regex=False), name] = 1

---

# Weather data

## Import Data

In [96]:
df_weather_code = pd.read_excel(os.path.join(root, 'weather_code.xlsx'), header=None, index_col=0)

In [97]:
weather_list = [pd.read_csv(os.path.join(root, 'weather_raw_20151228-20161227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'weather_raw_20161228-20171227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'weather_raw_20171228-20181227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'weather_raw_20181228-20191227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'weather_raw_20191228-20201227.csv'), encoding='euc-kr') , 
                pd.read_csv(os.path.join(root, 'weather_raw_20201228-20210818.csv'), encoding='euc-kr')]

## Preprocess
- '지점'에 따른 나라명 컬럼 (country) 추가
- 각 나라, 일자 별로 평균 강수량, 풍속, 기온 계산

In [98]:
# 지점에 따라 나라명 추가
def set_country(row):
    data = df_weather_code[df_weather_code[1] == row['지점']]
    if data.empty:
        return ""
    return data.iloc[0][2]


def preprocess_weather(df_weather):
    # 날짜 정보 정리
    df_weather['year'] = df_weather['일시'].astype('str').str[:4].astype('int')
    df_weather['month'] = df_weather['일시'].astype('str').str[5:7].astype('int')
    df_weather['day'] = df_weather['일시'].astype('str').str[8:10].astype('int')
    # 1차 평균
    df_weather['rain'] = df_weather[['지점', 'year', 'month', 'day', '강수량']].groupby(['지점', 'year', 'month', 'day']).transform('mean')
    df_weather['wind'] = df_weather[['지점', 'year', 'month', 'day', '풍속']].groupby(['지점', 'year', 'month', 'day']).transform('mean')
    df_weather['temperature'] = df_weather[['지점', 'year', 'month', 'day', '기온']].groupby(['지점', 'year', 'month', 'day']).transform('mean')
    # 컬럼/행 정리
    df_weather.drop(columns = ['지점명', '일시', '강수량', '풍속', '기온'], inplace=True)
    df_weather.drop_duplicates(inplace=True)
    # 나라명 추가
    df_weather['country'] = ""
    for i, row in df_weather.iterrows():
        df_weather.at[i, 'country'] = set_country(row)
    # 2차 평균
    df_weather['rain'] = df_weather[['country', 'year', 'month', 'day', 'rain']].groupby(['country', 'year', 'month', 'day']).transform('mean')
    df_weather['wind'] = df_weather[['country', 'year', 'month', 'day', 'wind']].groupby(['country', 'year', 'month', 'day']).transform('mean')
    df_weather['temperature'] = df_weather[['country', 'year', 'month', 'day', 'temperature']].groupby(['country', 'year', 'month', 'day']).transform('mean')
    # 컬럼/행 정리
    df_weather.drop(columns = ['지점'], inplace=True)
    df_weather.drop_duplicates(inplace=True)
    # 인덱스 정리
    df_weather.reset_index(drop=True, inplace=True)

In [99]:
for df in weather_list:
    preprocess_weather(df)
df_weather = pd.concat(rwt_list)

In [107]:
df_weather.describe()

,year,month,day,rain,wind,temperature
count,13947.000000,13947.000000,13947.000000,9561.000000,11895.000000,11895.000000
mean,2018.258407,6.276762,15.697426,-20.158210,3.535292,17.914166
std,1.613987,3.414373,8.809565,79.127632,5.846954,9.676148
min,2015.000000,1.000000,1.000000,-999.000000,-156.116667,-10.577841
25%,2017.000000,3.000000,8.000000,-6.661111,2.168701,11.250038
50%,2018.000000,6.000000,16.000000,1.165833,2.958333,20.135000
75%,2020.000000,9.000000,23.000000,3.915909,5.768750,25.950781
max,2021.000000,12.000000,31.000000,915.000000,15.125000,33.145833


#### 확인필요
- outlier 찾아내기 > 값이 너무 크거나 작은 경우 제외
- 날짜별로 확인 후 비어있는 값 채워넣기 (전/다음날 이용)
    - 13947개 데이터 중 rain, wind, temperature 갯수 보면 몇개 비어있는지 확인 가능
- 합치기..
- 강수량의 경우 NaN 값이 너무 많음
    - 위 전처리들을 한 후에도 많다면 사용불가
    - 위 전처리 후에는 적다면 전날/다음날 데이터 기반으로 채워넣기 진행

## Add to Training Data

In [ ]:
# 
def set_rwt(row):
    country = df_weather_code[df_weather_code[2] == row['CTRY_1']]
    if country.empty:
        return np.NaN
    data = df_weather[(df_weather['year'] == row['year']) 
                      & (df_weather['month'] == row['month']) 
                      & (df_weather['day'] == row['day'])
                      & (df_weather['지점'] == country[1])]
    return

In [ ]:
df_train['rain'] = np.NaN
df_train['wind'] = np.NaN
df_train['temperature'] = np.NaN

---

# Salinity

## Import Data

In [154]:
# df_salinity = pd.read_csv(os.path.join(root, 'salinity_raw.csv'))

## Drop Columns & Rows

In [155]:
# df_salinity.drop(df_salinity.columns[2], inplace=True, axis=1)
# df_salinity = df_salinity[(2015 <= df_salinity['obs_year']) & (df_salinity['obs_year'] <= 2021)]

2020, 2021 데이터의 부재로 인해 보류

---

# Oil

## Import Data

In [157]:
df_oil = pd.read_csv(os.path.join(root, 'oil_raw.csv'))
df_oil_dubai = pd.read_csv(os.path.join(root, 'oil_raw_dubai.csv'))
df_oil_brent = pd.read_csv(os.path.join(root, 'oil_raw_brent.csv'))

In [163]:
df_oil['날짜'].describe

<bound method NDFrame.describe of 0       2021년 06월 28일
1       2021년 06월 25일
2       2021년 06월 24일
3       2021년 06월 23일
4       2021년 06월 22일
            ...      
1446    2016년 01월 04일
1447    2015년 12월 31일
1448    2015년 12월 30일
1449    2015년 12월 29일
1450    2015년 12월 28일
Name: 날짜, Length: 1451, dtype: object>

In [182]:
def preprocess_oil(df):
    df['year'] = df['날짜'].str[:4]
    df['month'] = df['날짜'].str[6:8]
    df['day'] = df['날짜'].str[10:12]
    df['date'] = pd.to_datetime(df['year'] + df['month'] + df['day'])
    df = df.query('date.dt.dayofweek == 0')
    print(df.head())
    df.sort_index(ascending=False)

In [183]:
preprocess_oil(df_oil)

               날짜     종가     오픈     고가     저가      거래량    변동 %  year month  \
0   2021년 06월 28일  72.91  73.99  74.45  72.62  354.70K  -1.54%  2021    06   
5   2021년 06월 21일  73.66  71.52  73.96  71.15  123.66K   2.82%  2021    06   
10  2021년 06월 14일  70.88  70.65  71.78  70.65  319.64K  -0.04%  2021    06   
15  2021년 06월 07일  69.23  69.52  70.00  68.93  365.62K  -0.56%  2021    06   
20  2021년 05월 31일  66.96  66.58  67.41  66.41        -   0.56%  2021    05   

   day       date  
0   28 2021-06-28  
5   21 2021-06-21  
10  14 2021-06-14  
15  07 2021-06-07  
20  31 2021-05-31  


In [180]:
df_oil

,날짜,종가,오픈,고가,저가,거래량,변동 %,year,month,day,date
0,2021년 06월 28일,72.91,73.99,74.45,72.62,354.70K,-1.54%,2021,06,28,2021-06-28
1,2021년 06월 25일,74.05,73.32,74.18,72.85,364.72K,1.02%,2021,06,25,2021-06-25
2,2021년 06월 24일,73.30,73.28,73.61,72.32,334.83K,0.30%,2021,06,24,2021-06-24
3,2021년 06월 23일,73.08,72.91,74.25,72.82,429.21K,0.03%,2021,06,23,2021-06-23
4,2021년 06월 22일,73.06,73.41,73.95,72.94,29.00K,-0.81%,2021,06,22,2021-06-22
...,...,...,...,...,...,...,...,...,...,...,...
1446,2016년 01월 04일,36.76,37.60,38.39,36.33,431.99K,-0.76%,2016,01,04,2016-01-04
1447,2015년 12월 31일,37.04,36.81,37.79,36.22,283.62K,1.20%,2015,12,31,2015-12-31
1448,2015년 12월 30일,36.60,37.36,37.40,36.40,267.91K,-3.35%,2015,12,30,2015-12-30
1449,2015년 12월 29일,37.87,36.70,37.94,36.66,241.95K,2.88%,2015,12,29,2015-12-29


---

# Final

## onehot encoding

In [ ]:
one_hot = ['CTRY_1', 'CTRY_2', 'P_PURPOSE', 'CATEGORY_1', 'CATEGORY_2', 'P_NAME']

In [ ]:
df_train = pd.get_dummies(df_train, columns=one_hot)

## drop columns

In [ ]:
drop = ['REG_DATE', 'P_TYPE', 'P_IMPORT_TYPE']

In [ ]:
df_train.drop(columns = drop, inplace=True)

## Save files

# Train

## Set Metric

In [108]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

# Function to print the MAE (Mean Absolute Error) score
def print_score(m : LinearRegression):
    res = ['mae train: ', mean_absolute_error(m.predict(X_train), y_train), 
           'mae val: ', mean_absolute_error(m.predict(X_valid), y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

## Set Data

In [109]:
target = df_train['P_PRICE']
df_train.drop(columns = 'P_PRICE', inplace=True)

In [110]:
# x_train, x_test, y_train, y_test = train_test_split(df_train, target, random_state = 0)

NameError: name 'train_test_split' is not defined

In [ ]:
#Standardization
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

# Function for splitting training and validation data
def split_vals(a, n : int): 
    return a[:n].copy(), a[n:].copy()

val_perc = 0.1 # % to use for validation set
n_valid = int(val_perc * 100000) 
n_trn = len(df_train)-n_valid

# Split data
raw_train, raw_valid = split_vals(df_train, n_trn)
X_train, X_valid = split_vals(df_train, n_trn)
y_train, y_valid = split_vals(target, n_trn)

X_train=sc.fit_transform(X_train)
X_valid=sc.transform(X_valid)

# Check dimensions of samples
print('Sample train shape: ', X_train.shape, 
      '\nSample target shape: ', y_train.shape, 
      '\nSample validation shape: ', X_valid.shape)

- metric 선정하기
- base model 선정 > 전처리 최소화 + linear regression model 로 정확도 구하기
- 모델마다 최적화하기
- 모델끼리 비교하기

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostRegressor

In [ ]:
model = RandomForestRegressor(n_estimators=500,random_state=0)
model.fit(x_train,y_train)
cross = cross_val_score(model,x_test,y_test,cv = 5)

print('cross_tree',cross.mean())

In [ ]:
from sklearn.svm import SVR
model = SVR()
model.fit(x_train,y_train)
cross_svr = cross_val_score(model,x_test,y_test,cv = 5)
print('cross_svr',cross_svr.mean())

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=500,learning_rate=0.05,max_depth=3)
model.fit(x_train,y_train)
cross_xgboost = cross_val_score(model,x_test,y_test,cv = 5)

print('cross_xgboost : ',cross_xgboost.mean())

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(x_train,y_train)

cross_linear = cross_val_score(model,x_test,y_test,cv = 5)
print(cross_linear)
print('cross_linear : ',cross_linear.mean())

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor().fit(x_train,y_train)
cross_MLP = cross_val_score(model,x_test,y_test,cv = 5)

print('croos_MLP : ',cross_MLP.mean())